In [2]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt("Audiobooks_data.csv", delimiter=',')

### Extract the input from the dataset

In [7]:
unscaled_inputs = raw_csv_data[:,1:-1]

In [4]:
target = raw_csv_data[:,-1]

### Dataset Balancing 

In [5]:
# In building a machine learning model, balancing the dataset in very important
# ... so as to prevent us from have low accuracy

In [6]:
num_one_target = int(np.sum(target))
zero_target_counter = 0
indices_to_remove = []

for i in range(target.shape[0]): #loop through each index in the target array
    if target[i] == 0: #check if the current target is equals zero
        zero_target_counter +=1
        if zero_target_counter > num_one_target:
            indices_to_remove.append(i)

In [8]:
unscaled_inputs_equal_priors = np.delete(unscaled_inputs, indices_to_remove, axis=0)

In [9]:
target_equal_priors = np.delete(target, indices_to_remove, axis=0)

In [11]:
unscaled_inputs_equal_priors

array([[1620.  , 1620.  ,   19.73, ..., 1603.8 ,    5.  ,   92.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,    0.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,  388.  ],
       ...,
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,    6.  ],
       [1674.  , 3348.  ,    7.99, ...,    0.  ,    0.  ,    0.  ],
       [1674.  , 3348.  ,    5.33, ...,    0.  ,    0.  ,    0.  ]])

### More Explanation
1. The loop iterates through the target array index-by-index.
2. Whenever a 0 is encountered, the zero_target_counter increases.
3. If zero_target_counter exceeds num_one_target (i.e there are more 0 than 1), the index of that extra 0 is saved in indices_to_remove.
4. After this a typical removal of the extra rows of the 0s is carried out in both the target and the input(features) as seen in line 8 & 9.

### Standardization  of the input

In [12]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Performing Data Shuffling

In [13]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_target = target_equal_priors[shuffled_indices]

### Data splitting

In [16]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

In [21]:
train_inputs = shuffled_inputs[:train_samples_count]
train_target = shuffled_target[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_target = shuffled_target[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_target = shuffled_target[train_samples_count+validation_samples_count:]

In [22]:
print(np.sum(train_target), train_samples_count, np.sum(train_target)/train_samples_count)
print(np.sum(validation_target), validation_samples_count, np.sum(validation_target)/validation_samples_count)
print(np.sum(test_target), test_samples_count, np.sum(test_target)/test_samples_count)

1801.0 3579 0.5032131880413523
226.0 447 0.5055928411633109
210.0 448 0.46875


This code prints statistics about the distribution of class labels (0s and 1s) in the training, validation, and test datasets

In [23]:
np.savez("Audiobooks_data_train", inputs = train_inputs, targets = train_target)
np.savez("Audiobooks_data_validation", inputs = validation_inputs, targets = validation_target)
np.savez("Audiobooks_data_test", inputs = test_inputs, targets = test_target)

### Creating the machine learning algorithm

In [24]:
import tensorflow as tf

#### Loading the npz Files

In [28]:
npz = np.load("Audiobooks_data_train.npz")

train_inputs = npz["inputs"].astype(np.float32)
train_target = npz["targets"].astype(np.int32)

npz = np.load("Audiobooks_data_validation.npz")

validation_inputs, validation_target = npz["inputs"].astype(np.float32), npz["targets"].astype(np.int32)

npz = np.load("Audiobooks_data_test.npz")

test_inputs, test_target = npz["inputs"].astype(np.float32), npz["targets"].astype(np.int32)

#### Model

In [45]:
input_size = 10
output_size = 2
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(output_size, activation="softmax")
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Model Fitting and Early stopping mechanism 

In [46]:
batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_target, 
         batch_size=batch_size,
         epochs=max_epochs,
         callbacks = [early_stopping],
         validation_data=(validation_inputs, validation_target),
         verbose=2)

Epoch 1/100
36/36 - 2s - loss: 0.4967 - accuracy: 0.7365 - val_loss: 0.4246 - val_accuracy: 0.7673 - 2s/epoch - 54ms/step
Epoch 2/100
36/36 - 0s - loss: 0.3841 - accuracy: 0.7888 - val_loss: 0.4226 - val_accuracy: 0.7875 - 176ms/epoch - 5ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3623 - accuracy: 0.8050 - val_loss: 0.4150 - val_accuracy: 0.7785 - 175ms/epoch - 5ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3530 - accuracy: 0.8030 - val_loss: 0.3651 - val_accuracy: 0.8143 - 174ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3452 - accuracy: 0.8039 - val_loss: 0.3622 - val_accuracy: 0.8166 - 174ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3379 - accuracy: 0.8114 - val_loss: 0.3888 - val_accuracy: 0.8098 - 171ms/epoch - 5ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3354 - accuracy: 0.8164 - val_loss: 0.3785 - val_accuracy: 0.8166 - 230ms/epoch - 6ms/step


### Testing the model

In [47]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_target)

14/14 [==============================] - 0s 3ms/step - loss: 0.3537 - accuracy: 0.8125


In [49]:
print(f"Test loss {test_loss} and the test accuracy {test_accuracy*100}%")

Test loss 0.3537352383136749 and the test accuracy 81.25%
